In [19]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import os
import json
from tqdm import tqdm

In [20]:
# Load environment variables from .env file
load_dotenv()

# Set the API key from environment variables
# openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
)

In [21]:
def generate_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        # response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "Actua como un juez de derecho de restitucion de tierras"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0
    )
    return response.choices[0].message.content

In [22]:
# Load the JSON file back into a dictionary
with open('filtered_dict.json', 'r') as json_file:
    loaded_dict = json.load(json_file)

# print("Loaded Dictionary from JSON:")
# print(loaded_dict)

file_list = list(loaded_dict.keys())

# Save the file list to a new JSON file
with open('file_list.json', 'w') as json_file:
    json.dump(file_list, json_file, indent=4)

# Read the list of file names from the JSON file
with open('file_list.json', 'r') as json_file:
    file_list = json.load(json_file)


In [23]:
results = {}

# Create the directory to save the responses if it doesn't exist
output_directory = './responses'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Track progress in a txt file
progress_file = 'progress.txt'
if os.path.exists(progress_file):
    with open(progress_file, 'r') as file:
        completed_files = {line.strip() for line in file}
else:
    completed_files = set()


In [ ]:
instructions=[
"""Instrucciones: Quiero sacar la siguiente informacion del texto anterior. En caso de no encontrar la informacion responder ‘NULL’.
1. Nombre del Juzgado, Tribunal o Juez que decide el caso 
2. Radicado de la sentencia
3. Fecha de la sentencia (responder en formato mes/día/año)
4. Fecha en la se solicito el predio en restitución (responder en formato mes/día/año)
5. Fecha en la que se inscribió el predio al Registro Único de Tierras Despojadas y Abandonas Forzosamente (responder en formato mes/día/año)
6. El número de demandantes
7. El numero de cedula o de identidad de los demandantes (si es mas de uno separarlos por el caracter “\”)
8. El número total de predios o parcelas demandadas
9. El número total de predios o parcelas restituidos
10. Departamento
11. Municipio
12. Vereda
13. Numero de personas heridas en el proceso de desposesion (si no hay responder 0)
14. Numero de personas muertas en el proceso de desposesion (si no hay responder 0)
15. Hubo desplazamiento forzado de los demandantes despues de la desposesion? (responder si o no)
16. Los demandantes perdieron el predio debido a que razon? (elijir de las siguientes categorias: violencia en la zona, violencia directa - si es mas de uno separarlos por el caracter “\”))

Detalles: Como output quiero que me devuelva un formato separado por punto y comas. No quiero el archivo csv solo el texto generado. Tampoco quiero los titulos de las columnas, solamente sus respuestas. Este es el formato que quiero:
var1; var2; var3; var4; var5; var6; var7; var8; var9; var10; var11; var12; var13; var14; var15; var16; var17
""",
"""Instrucciones: Quiero sacar la siguiente informacion del anterior texto, pero enfocandome en el rol de los opositores. En caso de no encontrar la informacion responder ‘NULL’.
1.Radicado de la sentencia
2. El numero de opositores (en caso de que no hayan opositores responder 0 y ‘NULL’ en las siguientes preguntas)
3. Nombres de los opositores (si es mas de uno separarlos por el caracter “\”)
4. Los opositores probaron/acreditaron buena fe exenta de culpa? (responder si o no) Esto se puede extraer del la parte final del documento.
5. Los opositores recibieron alguna compensacion economica? (responder si o no)
6. Hay pruebas de que los opositores ejercieron directamente violencia/presion/amenazas sobre los demandantes? (responder si o no)
7. Los opositores estan relacionados con politica, paramilitares, guerrilla, narcotrafico? (responder si o no)
8. Si los opositores estan relacionados con algun actor, escoger politica, paramilitares, guerrilla, narcotrafico. De lo contrario responder ‘NULL’
9.Con que actividad productiva estan relacionados los opositores? (escoger entre ganaderia, agricultura, mineria, construccion, ninguno)
10. El numero de cedula o de identidad de los opositores (si es mas de uno separarlos por el caracter “\”)

Detalles: Como output quiero que me devuelva las respuestas en un formato separado por punto y comas. No quiero el archivo csv solo el texto generado. Tampoco quiero los titulos de las columnas, solamente sus respuestas. Este es el formato que quiero:
var1; var2; var3; var4; var5; var6; var7; var8; var9; var10
Si no hay opositores hacer una fila con `NULL' para cada pregunta.
""",
"""
Instrucciones: Para CADA predio demandado quiero sacar la siguiente informacion del anterior texto. En caso de no encontrar la informacion responder ‘NULL’.
1. Numero de matricula inmobiliaria
2. Radicado de la sentencia 
3. Numero de cédula catastral
4. Nombre del predio demandado
5. El juez decidio restituir el predio/parcela? (responder si o no)
6. Si se restituyo el predio, fue: el mismo, equivalente, dinero (escoger solo una categoria. En caso de que no fuera restituido responder `NULL'
7. Si se restituyo el predio, se dio la propiedad sobre el? (responder si o no)
8. Tamano/area en hectareas (Solo en numero no quiero texto)
9. El predio es rural? (responder si o no)
10. Relacion del demandante con el predio? (elija alguna de las siguientes categorias: propiedad, ocupacion, arriendo, herencia, prescripcion, adjudicacion, otro)
11. Que uso se le daba al predio antes de la desposesion? (ganaderia, agricultura, mineria, construccion, ninguno)
12. Fecha en al que el demandante perdio el predio
13. La posesion se perdio porque razon? (elija de las siguientes categorias: violencia en la zona, violencia directa - si es mas de uno separarlos por el caracter “\”)
14. Si hubo violencia en la zona, de que clase fue? (elija de las siguientes categorias: masacre, enfrentamientos con el ejercito, enfrentamientos con otro grupo armado, limpieza social, advertencia o anuncio general, asesinatos selectivos - si es mas de uno separarlos por el caracter “\”)
15. Si hubo violencia directa, de que clase fue? (elija de las siguientes categorias: amenaza, muerte de familiar o conocido, lesion fisica, intento de homicidio, tortura - si es mas de uno separarlos por el caracter “\”)
16. La violencia fue ejercida por: paramilitares, guerrilla, narcotrafico, bandas criminales, ejercito (si es mas de uno separarlos por el caracter “\”)
17. Nombre del grupo armado, empresa, entidad, o persona que ejercio violencia sobre los demandantes (si es mas de uno separarlos por el caracter “\”)
18. Nombres de paramilitares, guerrilleros o politicos relacionados con el caso (si es mas de uno separarlos por el caracter “\”)
19. La posesion se perdio por medio de: venta forzada, venta voluntaria, donacion, adjudicacion, abandono, despojo (si es mas de uno separarlos por el caracter “\”)
20. Si hubo venta, cuanto dinero les dieron a cambio por este predio? (respuesta en COP, si no hubo venta responder ‘NULL’)
21. Que uso se le da al predio despues de la desposesion? (ganaderia, agricultura, mineria, construccion, ninguno)
Detalles: Como output quiero que me devuelva un formato separado por punto y comas. No quiero el archivo csv solo el texto generado. Tampoco quiero los titulos de las columnas, solamente sus respuestas. Este es el formato que quiero:
Numero de matricula inmobiliaria 1; Radicado de la sentencia; var3; var4; var5; var6; var7; var8; var9; var10; var11; var12; var13; var14; var15; var16; var17; var18; var19; var20; var21
Numero de matricula inmobiliaria 2; Radicado de la sentencia; var3; var4; var5; var6; var7; var8; var9; var10; var11; var12; var13; var14; var15; var16; var17; var18; var19; var20; var21
Numero de matricula inmobiliaria 3; Radicado de la sentencia; var3; var4; var5; var6; var7; var8; var9; var10; var11; var12; var13; var14; var15; var16; var17; var18; var19; var20; var21
"""
]

In [25]:
try:
    for file_name in tqdm(file_list):
        if file_name in completed_files:
            continue
        print('----------file name------', file_name)
        # Construct the file path
        file_path = os.path.join('./processed_files', file_name)
        
        # Read the content of the file
        with open(file_path, 'r') as file:
            context = file.read()
        
        # Generate responses for each instruction
        answer_opositores = ""
        answers = {}
        for i in range(len(instructions)):
            prompt = f"""Actua como un juez de acuerdo de restitucion de tierras
            texto:
            {context}
            a partir del anterior texto ejecute las siguientes instucciones:
            {instructions[i]}
            solo responda en formato csv con formato separado por  ; nada mas
            """
            response = generate_response(prompt)
            if i == 0:
                answer_opositores = response
            if (i == 2 or i == 3) and (answer_opositores == "0" or answer_opositores == "NULL"):
                continue
            answers[f'answers_prompt_{i}'] = response
        
        # Save the answers in the results dictionary
        results[file_name] = answers

        # Save the response for the current file in a new JSON file
        output_path = os.path.join(output_directory, f"{file_name}.json")
        with open(output_path, 'w') as json_file:
            json.dump(answers, json_file, indent=4)

        # Update the progress file
        with open(progress_file, 'a') as file:
            file.write(f"{file_name}\n")

except Exception as e:
    print(f"An error occurred: {e}")

print("Processing complete. Results have been saved.")


  0%|          | 0/1437 [00:00<?, ?it/s]

----------file name------ 86001312140120180000501 Puerto Guzman 31 marzo 2022.txt


  1%|          | 11/1437 [00:09<19:31,  1.22it/s]

----------file name------ 76001312100120200007600 Tulua 24 marzo 2022.txt


  1%|          | 12/1437 [00:14<33:31,  1.41s/it]

...

----------file name------ 20001312100120190009501 Agustin Codazzi 29 abril 2022.txt


  1%|▏         | 21/1437 [03:45<7:18:41, 18.59s/it]

----------file name------ 20001312100320210005600 Agustin Codazzi 30 septiembre 2021.txt


  1%|▏         | 21/1437 [04:00<4:30:48, 11.47s/it]


KeyboardInterrupt: 

In [31]:
(5/1000000*40000*3 + 15/1000000*500)*1437

872.9775000000001